![BAE logo](images/bae_logo.png)

# Hands-On Exercise 13.1: Asynchronous Keywords

## Objective

The goal of this exercise is to become familiar with using the asynchronous keywords in C# and to fix compute-bound thread that locks and blocks the user interface.

This exercise returns to the Blackjack simulation we used earlier in the course. There is a problem. Using Monte Carlo techniques to test certain game features requires many samples for the statistical result to be accurate. This unfortunately causes the user interface to hang.

#### Open the starting point for this exercise.

Close the previous solution if you have not already done so.

Open the solution Blackjack located in the directory `C:\Course\510D\Exercises\Ex131` and test the behavior.
    
A "deal tester" is used to check that the card deck shuffling is accurate.

Open `DealTester.cs` (in the `CardLib` project) for editing and review it to see what it does.
    
If the deck's shuffling method is working correctly and we have enough statistical samples, what average should be returned to prove that it is working correctly?

For display to the user, the average will be multiplied by 4 to get an overall percentage of accuracy for all suits.

#### Answer...

Hearts should be drawn 25% of the time (so 100% once it's multiplied by 4).

Execute the program. Select the **Simulation | Test Card Dealing** menu item.

The system will hang waiting for the test to execute—5,000,000 shuffle/ deal loops. It will take about 15 seconds to complete.

Eventually an average of 100% should be shown in the status bar at the bottom of the game's display.

To get statistical accuracy, many cards must be dealt. However, this compute-bound code has adverse effects on the overall application. It tends to lock it up—it cannot even exit. Fixing this is a job for threading!

#### Change the `dealTestingToolStripMenuItem_click(...)` method to be asynchronous.

Open the file `MainGameTable.cs` for editing and at the top, confirm the statement `using System.Threading;` is there.

After it add `using.Threading.Tasks`.

Find the method `dealTestingToolStripMenuItem_click(...)` and add the keyword `async` to its declaration.

You can use:

```C#
private async void dealTestingToolStripMenuItem_Click(object sender, EventArgs e)
```

Add a line to report that the test was started. Make this the first line of code in the method.

```C#
StatusLabel.Text = "Deal tester started.";
```

Continue editing within this method. After the line of code that creates the `DealTester` object, create an `Action` called `deal` that takes `tester.Deal` as its delegated method.

You can use:

```C#
Action deal = new Action(tester.Deal);
```

In the next line, start the deal testing by calling `Task.Run(deal)`. Add the keyword to make this asynchronous.

You can use:

```C#
await Task.Run(deal);
```

Comment out the line of code that contains `tester.Deal();` since this has been replaced with the asynchronous call.

Run the program.
    
Confirm that you can deal cards while the deal tester is running 5 million times. Also check that the tester reports 100% after about 15 seconds or so.

Whenever asynchronous operations are done, it is important to prevent race conditions. The user can start multiple deal testers at the same time. This should be prevented.

#### Prevent multiple invocations of deal tester.

Set the deal tester menu item to disabled before the asynchronous operation and re-enable it after the asynchronous operation has finished.

Disable the deal tester menu adding `dealTestingToolStripMenuItem.Enabled = false;` and insert it to be the first line of code in the method.

Set the menu back to enabled once the deal testing is finished. Make it the last line of code in the method.

You can use:

```C#
dealTestingToolStripMenuItem.Enabled = true;
```

Compile and test. Ensure that the Test Card Dealing menu item is disabled while the test is running. Test multiple threads at once by:

- Start the deal tester
- Change the color of the screen
- Play the game

The game should no longer hang when deal testing is being done.

Run all tests to ensure we have not broken anything.

## Congratulations! You have successfully completed the exercise. User interface responsiveness has been improved. Continue to the bonus, if you have more time.

# Bonus (Optional)

Add a timer to monitor and report on the status of the dealing test thread.

In the application as currently written, the user cannot tell if the dealing test has failed or is just taking a long time to finish. Let's add a timer to monitor the dealing test thread and output a "thermometer" while it is running!

#### Add a timer to the form.

Open `MainGameTable.cs` in designer mode. Drag and drop a timer control to the form. It should appear at the bottom of the designer window as `timer1`` In the timer's property sheet, set its interval to be 500 milliseconds.

Get a clean compile.

#### Start and stop the progress timer (timer1), and set the progress bar to be visible as appropriate when the dealing test starts and stops.

Return to editing `MainGameTable.cs` and, in the `dealTestingToolStripMenuItem_Click` method after the line of code that reports the deal tester started, add logic to:

- Set `ProgressBar` to visible
- Set the progress bar's maximum to be the number of loops that the test will run (`5000000`)
- Set the progress bar's minimum to `0`
- Set the progress bar's value to be `0`
- Start the timer (`timer1`)

Move to before the line of code that reports the average and add the logic to stop the timer (`timer1`) and set the progress bar to not visible.

Compile and test. The progress bar should appear when the deal testing is started and disappear when finished. It will not yet show the progress changing.

#### Implement the timer's event method.

View `MainGameTable` in designer mode and double-click `timer1` to create its event method.

In the body of `timer1_Tick`, set the value of the progress bar to the number of loops executed. The code should look something like:

```C#
ProgressBar.Value = tester.LoopsCompleted;
```

Compile and test. The "thermometer" should now show the status of the dealing test. Note that you can continue to play the game. Try fading (changing) the background color preferences at the same time—two timers at once.

Run all tests to ensure you still have green lights.

## Congratulations! You have completed the bonus.